In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clean_raw_data.csv
/kaggle/input/clean_data_mean.csv
/kaggle/input/raw_data.csv
/kaggle/input/clean_data_GCN.csv
/kaggle/input/FillZero_minmax_baseline/clean_data_minmax_fill-zero.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/val/val_week1_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/test/test_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/clean_data_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_4.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_1.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_5.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/val/val_week1_2_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/test/test_week3.csv
/kaggle/input/FillZero_minmax_bas

In [2]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import StratifiedKFold
import time
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_auc_score

<ipython-input-2-63192117ceec>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
# Biến global cho base path
BASE_PATH = "/kaggle/input/GCN_minmax_baseline_version4"
# Tuần và số phần fold
weeks = ['week1', 'week2', 'week3', 'week4']
fold_parts = 5

# Tạo five_fold_files
five_fold_files = {
    week: [
        f"{BASE_PATH}/clean_{week}/train/5-folds/data_part_{i}.csv"
        for i in range(1, fold_parts + 1)
    ]
    for week in weeks
}

# Tạo file_validation
file_validation = {
    'week1': [f"{BASE_PATH}/clean_week1/val/val_week1.csv"],
    'week2': [f"{BASE_PATH}/clean_week2/val/val_week1_2.csv"],
    'week3': [f"{BASE_PATH}/clean_week3/val/val_week1_2_3.csv"],
    'week4': [f"{BASE_PATH}/clean_week4/val/val_week1_2_3_4.csv"]
}

# Tạo file_test
file_test = {
    week: [f"{BASE_PATH}/clean_{week}/test/test_{week}.csv"]
    for week in weeks
}


In [4]:
# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    return focal_loss_fixed

# Tạo hàm train cho từng tuần
def train_week_model(week_number, file_paths_train, file_validataion):
   # Đọc dữ liệu
    train_data = pd.read_csv(file_paths_train)
    val_data = pd.read_csv(file_validataion)
    
    # Tách đặc trưng và nhãn
    X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_train = train_data["classification_encoded"]

    X_val = val_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_val = val_data["classification_encoded"]
    

    # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
    oversampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)

    # One-hot encode nhãn
    y_train_res = tf.keras.utils.to_categorical(y_train_res, num_classes=5)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)

    # Xây dựng mô hình 4-layer stacked LSTM
    def build_model(hp):
        model = tf.keras.Sequential()
        
        # LSTM layer 1
        model.add(layers.LSTM(
            units=hp.Int('units_1', min_value=32, max_value=256, step=32),
            return_sequences=True,
            input_shape=(X_train_res.shape[1], 1)
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))
        
        # LSTM layer 2
        model.add(layers.LSTM(
            units=hp.Int('units_2', min_value=32, max_value=256, step=32),
            return_sequences=True
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))
        
        # LSTM layer 3
        model.add(layers.LSTM(
            units=hp.Int('units_3', min_value=32, max_value=256, step=32),
            return_sequences=True
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_3', min_value=0.1, max_value=0.5, step=0.1)))
        
        # LSTM layer 4
        model.add(layers.LSTM(
            units=hp.Int('units_4', min_value=32, max_value=256, step=32),
            return_sequences=False
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_4', min_value=0.1, max_value=0.5, step=0.1)))

        # Lớp đầu ra
        model.add(layers.Dense(5, activation='softmax'))

        # Compile với Focal Loss
        model.compile(optimizer=tf.keras.optimizers.Adam(
                          learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss=focal_loss(gamma=2., alpha=0.25),
                      metrics=['accuracy'])
        return model

    # Khởi tạo RandomSearch tuner
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=10,
        executions_per_trial=1,
        directory='my_dir',
        project_name=f'lstm_tuning_week{week_number}'
    )

    # Tìm kiếm siêu tham số tốt nhất
    tuner.search(X_train_res, y_train_res,
                 epochs=20,
                 validation_data=(X_val, y_val),
                 batch_size=32)

    # Trả về kết quả tối ưu cho tuần
    best_params = tuner.get_best_hyperparameters(num_trials=1)[0]
    return best_params


In [5]:
# Định nghĩa đường dẫn đến dữ liệu cho từng tuần
file_paths_train = {
    week: f"{BASE_PATH}/clean_{week}/train/clean_data_{week}.csv"
    for week in weeks
}

# Định nghĩa file_validation theo quy luật riêng
file_validation = {
    f"week{idx + 1}": f"{BASE_PATH}/clean_week{idx + 1}/val/val_week{'_'.join(str(i) for i in range(1, idx + 2))}.csv"
    for idx in range(len(weeks))
}

In [6]:
# Tìm tham số tốt nhất cho từng tuần
best_params_week1 = train_week_model(1, file_paths_train["week1"], file_validation["week1"])
best_params_week2 = train_week_model(2, file_paths_train["week2"], file_validation["week2"])
best_params_week3 = train_week_model(3, file_paths_train["week3"], file_validation["week3"])
best_params_week4 = train_week_model(4, file_paths_train["week4"], file_validation["week4"])

# In thông tin chi tiết các tham số tối ưu
print("Best Parameters for Week 1:")
for param_name in best_params_week1.values.keys():
    print(f"{param_name}: {best_params_week1.get(param_name)}")

print("\nBest Parameters for Week 2:")
for param_name in best_params_week2.values.keys():
    print(f"{param_name}: {best_params_week2.get(param_name)}")

print("\nBest Parameters for Week 3:")
for param_name in best_params_week3.values.keys():
    print(f"{param_name}: {best_params_week3.get(param_name)}")

print("\nBest Parameters for Week 4:")
for param_name in best_params_week4.values.keys():
    print(f"{param_name}: {best_params_week4.get(param_name)}")


Trial 10 Complete [00h 04m 24s]
val_accuracy: 0.9938986897468567

Best val_accuracy So Far: 0.9938986897468567
Total elapsed time: 00h 44m 41s
Best Parameters for Week 1:
units_1: 32
dropout_1: 0.5
units_2: 32
dropout_2: 0.30000000000000004
units_3: 128
dropout_3: 0.2
units_4: 256
dropout_4: 0.1
learning_rate: 0.0017649674278631807

Best Parameters for Week 2:
units_1: 160
dropout_1: 0.2
units_2: 160
dropout_2: 0.5
units_3: 96
dropout_3: 0.5
units_4: 192
dropout_4: 0.1
learning_rate: 0.0014789571342550213

Best Parameters for Week 3:
units_1: 224
dropout_1: 0.30000000000000004
units_2: 192
dropout_2: 0.1
units_3: 128
dropout_3: 0.30000000000000004
units_4: 128
dropout_4: 0.30000000000000004
learning_rate: 0.0029973497945481892

Best Parameters for Week 4:
units_1: 64
dropout_1: 0.2
units_2: 96
dropout_2: 0.2
units_3: 192
dropout_3: 0.5
units_4: 256
dropout_4: 0.2
learning_rate: 0.005529023837534512


In [7]:
# Danh sách tham số tốt nhất
best_params = {
    "week1": best_params_week1,
    "week2": best_params_week2,
    "week3": best_params_week3,
    "week4": best_params_week4
}

In [8]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Xây dựng mô hình
def build_lstm_model(params):
    model = tf.keras.Sequential()
    
    # LSTM layer 1
    model.add(layers.LSTM(
        units=params.get('units_1', 32),
        return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))
    )
    model.add(layers.Dropout(rate=params.get('dropout_1', 0.2)))

    # LSTM layer 2
    model.add(layers.LSTM(
        units=params.get('units_2', 32),
        return_sequences=True
    ))
    model.add(layers.Dropout(rate=params.get('dropout_2', 0.2)))

    # LSTM layer 3
    model.add(layers.LSTM(
        units=params.get('units_3', 32),
        return_sequences=True
    ))
    model.add(layers.Dropout(rate=params.get('dropout_3', 0.2)))

    # LSTM layer 4
    model.add(layers.LSTM(
        units=params.get('units_4', 32),
        return_sequences=False
    ))
    model.add(layers.Dropout(rate=params.get('dropout_4', 0.2)))

    # Output layer
    model.add(layers.Dense(5, activation='softmax'))  # 5 classes in output

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
        loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
        metrics=['accuracy']
    )
    
    return model


In [9]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week].values
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "auc_roc_per_label": [],    # AUC từng lớp
        "auc_roc_macro": [],        # AUC macro
        "auc_roc_weighted": [],     # AUC weighted (tự tính)
        "precision_macro": [],
        "recall_macro": [],
        "f1_macro": [],
        "precision_weighted": [],
        "recall_weighted": [],
        "f1_weighted": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = to_categorical(train_data['classification_encoded'], num_classes=5)
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = to_categorical(test_data['classification_encoded'], num_classes=5)

        # Reshape dữ liệu cho LSTM
        X_train = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

        # Xây dựng mô hình với tham số tốt nhất
        model = build_lstm_model(params)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        week_results["accuracy_per_fold"].append(accuracy)
        
        # Dự đoán
        y_pred_classes = y_pred.argmax(axis=1)
        y_test_classes = y_test.argmax(axis=1)
        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
        conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
        
        # Tính AUC-ROC
        try:
            # Tính AUC macro và theo từng lớp với OvR
            auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
            auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
            # Tính AUC weighted: tính trọng số theo số mẫu của từng lớp
            supports = np.bincount(y_test_classes, minlength=5)
            auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
        except Exception as e:
            print(f"Lỗi khi tính AUC: {e}")
            auc_macro = np.nan
            auc_per_class = [np.nan] * 5
            auc_weighted = np.nan
            
        # Lưu kết quả của fold hiện tại
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["auc_roc_per_label"].append(auc_per_class)  # AUC từng lớp
        week_results["auc_roc_macro"].append(auc_macro)          # AUC macro
        week_results["auc_roc_weighted"].append(auc_weighted)      # AUC weighted
        week_results["confusion_matrices"].append(conf_matrix)
        week_results["precision_macro"].append(precision_macro)
        week_results["recall_macro"].append(recall_macro)
        week_results["f1_macro"].append(f1_macro)
        week_results["precision_weighted"].append(precision_weighted)
        week_results["recall_weighted"].append(recall_weighted)
        week_results["f1_weighted"].append(f1_weighted)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.nanmean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.nanmean(week_results["f1_score_per_label"], axis=0)
    average_auc_per_label = np.nanmean(week_results["auc_roc_per_label"], axis=0)
    average_confusion_matrix = np.nanmean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    average_accuracy = np.nanmean(week_results["accuracy_per_fold"])
    average_precision_macro = np.nanmean(week_results["precision_macro"])
    average_recall_macro = np.nanmean(week_results["recall_macro"])
    average_f1_macro = np.nanmean(week_results["f1_macro"])
    average_auc_macro = np.nanmean(week_results["auc_roc_macro"])
    average_precision_weighted = np.nanmean(week_results["precision_weighted"])
    average_recall_weighted = np.nanmean(week_results["recall_weighted"])
    average_f1_weighted = np.nanmean(week_results["f1_weighted"])
    average_auc_weighted = np.nanmean(week_results["auc_roc_weighted"])


    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test_classes)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label,
        "Average AUC": average_auc_per_label
    })
    
    # Tạo DataFrame cho confusion matrix
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    # In kết quả Accuracy và Macro metrics
    print("\n=== Average Accuracy ===")
    print(f"{average_accuracy:.4f}")
    print("\n=== Average Macro Metrics ===")
    print(f"Macro Precision: {average_precision_macro:.4f}")
    print(f"Macro Recall: {average_recall_macro:.4f}")
    print(f"Macro F1-Score: {average_f1_macro:.4f}")
    print(f"Macro AUC-ROC: {average_auc_macro:.4f}")
    print("\n=== Average Weighted Metrics ===")
    print(f"Weighted Precision: {average_precision_weighted:.4f}")
    print(f"Weighted Recall: {average_recall_weighted:.4f}")
    print(f"Weighted F1-Score: {average_f1_weighted:.4f}")
    print(f"Weighted AUC-ROC: {average_auc_weighted:.4f}")
    print("\n=== Average Metrics per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)
    
    # Cập nhật kết quả cho tuần hiện tại
    week_results.update({
        "average_accuracy": average_accuracy,
        "average_precision_macro": average_precision_macro,
        "average_recall_macro": average_recall_macro,
        "average_f1_macro": average_f1_macro,
        "average_auc_macro": average_auc_macro,
        "average_precision_weighted": average_precision_weighted,
        "average_recall_weighted": average_recall_weighted,
        "average_f1_weighted": average_f1_weighted,
        "average_auc_weighted": average_auc_weighted,
        "average_metrics_df": metrics_df,
        "average_confusion_matrix": confusion_df,
        "average_train_times": average_train_time,
        "average_test_times": average_test_time,
    })
    overall_results_5folds.append(week_results)


Processing week1 with best parameters...
best parameters for week1: {'units_1': 32, 'dropout_1': 0.5, 'units_2': 32, 'dropout_2': 0.30000000000000004, 'units_3': 128, 'dropout_3': 0.2, 'units_4': 256, 'dropout_4': 0.1, 'learning_rate': 0.0017649674278631807}
Fold 1: Using file /kaggle/input/GCN_minmax_baseline_version4/clean_week1/train/5-folds/data_part_1.csv as test set
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.6003 - loss: 0.1644 - val_accuracy: 0.6256 - val_loss: 0.1272
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6441 - loss: 0.1324 - val_accuracy: 0.6424 - val_loss: 0.1215
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6483 - loss: 0.1273 - val_accuracy: 0.6820 - val_loss: 0.1158
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6573 - loss: 0.1245 - val_accuracy: 0.6866 - val_loss: 0.1111
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.6607 - loss: 0.1234 - val_accuracy: 0.6958 - val_loss: 0.1092
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6744 - loss: 0.1224 - val_accuracy: 0.7205 - val_loss: 0.1052
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6717 - loss: 0.1171 - val_accuracy: 0.7160 - val_loss: 0.1064
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6805 - loss: 0.1184 - val_accuracy: 0.7099 - val_

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn

Fold 2: Using file /kaggle/input/GCN_minmax_baseline_version4/clean_week1/train/5-folds/data_part_2.csv as test set
Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.6078 - loss: 0.1632 - val_accuracy: 0.6142 - val_loss: 0.1334
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6130 - loss: 0.1354 - val_accuracy: 0.6271 - val_loss: 0.1228
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.6400 - loss: 0.1269 - val_accuracy: 0.6233 - val_loss: 0.1159
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6355 - loss: 0.1238 - val_accuracy: 0.6260 - val_loss: 0.1142
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6503 - loss: 0.1208 - val_accuracy: 0.7038 - val_loss: 0.1132
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6724 - loss: 0.1169 - val_accuracy: 0.7106 - val_loss: 0.1103
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6571 - loss: 0.1204 - val_accuracy: 0.7186 - val_

...

Fold 3: Using file /kaggle/input/GCN_minmax_baseline_version4/clean_week4/train/5-folds/data_part_3.csv as test set
Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.6063 - loss: 0.1479 - val_accuracy: 0.6304 - val_loss: 0.1233
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.6711 - loss: 0.1199 - val_accuracy: 0.6728 - val_loss: 0.1154
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7117 - loss: 0.1155 - val_accuracy: 0.6857 - val_loss: 0.1189
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6870 - loss: 0.1194 - val_accuracy: 0.7231 - val_loss: 0.1103
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7343 - loss: 0.1085 - val_accuracy: 0.7822 - val_loss: 0.0899
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7099 - loss: 0.1115 - val_accuracy: 0.6804 - val_loss: 0.1224
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6840 - loss: 0.1182 - val_accuracy: 0.8108 - val_l

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn

Fold 4: Using file /kaggle/input/GCN_minmax_baseline_version4/clean_week4/train/5-folds/data_part_4.csv as test set
Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.6012 - loss: 0.1471 - val_accuracy: 0.6323 - val_loss: 0.1236
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6507 - loss: 0.1224 - val_accuracy: 0.6609 - val_loss: 0.1237
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.6757 - loss: 0.1235 - val_accuracy: 0.6846 - val_loss: 0.1129
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7052 - loss: 0.1155 - val_accuracy: 0.7796 - val_loss: 0.0909
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7448 - loss: 0.1019 - val_accuracy: 0.8291 - val_loss: 0.0793
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7373 - loss: 0.1066 - val_accuracy: 0.8299 - val_loss: 0.0760
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7619 - loss: 0.0989 - val_accuracy: 0.8284 - val_

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn

Fold 5: Using file /kaggle/input/GCN_minmax_baseline_version4/clean_week4/train/5-folds/data_part_5.csv as test set
Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.6018 - loss: 0.1486 - val_accuracy: 0.6346 - val_loss: 0.1161
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6820 - loss: 0.1197 - val_accuracy: 0.7746 - val_loss: 0.0989
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7365 - loss: 0.1091 - val_accuracy: 0.7727 - val_loss: 0.0964
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7045 - loss: 0.1139 - val_accuracy: 0.6751 - val_loss: 0.1153
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.6570 - loss: 0.1237 - val_accuracy: 0.8322 - val_loss: 0.0776
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7220 - loss: 0.1099 - val_accuracy: 0.7818 - val_loss: 0.0926
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7412 - loss: 0.1069 - val_accuracy: 0.6541 - val_

In [10]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_accuracy = np.mean(week_results["accuracy_per_fold"])
    average_confusion_matrix = week_result["average_confusion_matrix"]
    
    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Accurancy: {average_accuracy}")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print(f"Average AUC Macro: {average_auc_macro}")
    print(f"Average AUC Weighted: {average_auc_weighted}")
    print("\nAverage Precision, Recall, F1-Score, AUC-ROC per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Accurancy: 0.8235916137695313
Average Train Time: 137.9115 seconds
Average Test Time: 0.7923 seconds
Average AUC Macro: 0.9095970693120643
Average AUC Weighted: 0.9582081303486953

Average Precision, Recall, F1-Score, AUC-ROC per Label:
   Label  Average Precision  Average Recall  Average F1-Score  Average AUC
0      0           0.664101        0.650333          0.656388     0.856706
1      1           0.137500        0.025000          0.042308     0.828631
2      2           0.539399        0.549897          0.541719     0.858736
3      3           0.647660        0.501796          0.564297     0.882926
4      4           0.840316        0.906817          0.872087     0.914220

Average Confusion Matrix:
       0    1     2     3       4
0  390.2  0.2  29.6  26.8   153.2
1   42.4  2.2   4.0   2.6    36.4
2   34.4  0.2  90.4   6.2    33.2
3   24.8  0.0   4.4  83.8    54.0
4   97.8  0.6  40.8  10.2  1454.0

=== Results for week2 ===
Average Accurancy: 0

## Kiểm tra trên tập test

In [11]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

def build_lstm_model(params):
    model = tf.keras.Sequential()
    
    # LSTM layer 1
    model.add(layers.LSTM(
        units=params.get('units_1', 32),
        return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))
    )
    model.add(layers.Dropout(rate=params.get('dropout_1', 0.2)))

    # LSTM layer 2
    model.add(layers.LSTM(
        units=params.get('units_2', 32),
        return_sequences=True
    ))
    model.add(layers.Dropout(rate=params.get('dropout_2', 0.2)))

    # LSTM layer 3
    model.add(layers.LSTM(
        units=params.get('units_3', 32),
        return_sequences=True
    ))
    model.add(layers.Dropout(rate=params.get('dropout_3', 0.2)))

    # LSTM layer 4
    model.add(layers.LSTM(
        units=params.get('units_4', 32),
        return_sequences=False
    ))
    model.add(layers.Dropout(rate=params.get('dropout_4', 0.2)))

    # Output layer
    model.add(layers.Dense(5, activation='softmax'))  # 5 classes in output

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
        loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
        metrics=['accuracy']
    )
    
    return model

In [12]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    params = best_params[f"week{week_num}"].values
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"{BASE_PATH}/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"{BASE_PATH}/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách X và y
    X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                       "course_id", "school", "enroll_time", "classification"])
    y_train = train_data['classification_encoded']
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                     "course_id", "school", "enroll_time", "classification"])
    y_test = test_data['classification_encoded']

    # Áp dụng SMOTE cho tập huấn luyện
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Chuyển đổi nhãn sang dạng one-hot
    y_train_resampled = to_categorical(y_train_resampled, num_classes=5)
    y_test = to_categorical(y_test, num_classes=5)

    # Reshape dữ liệu cho LSTM
    X_train_resampled = X_train_resampled.to_numpy().reshape((X_train_resampled.shape[0], 1, X_train_resampled.shape[1]))
    X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

    # Xây dựng mô hình với tham số tốt nhất
    model = build_lstm_model(params)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train_resampled, y_train_resampled, epochs=50, validation_split=0.1, batch_size=32)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    # Đánh giá mô hình
    y_pred_classes = y_pred.argmax(axis=1)
    y_test_classes = y_test.argmax(axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    
    # Tính AUC-ROC (với one-vs-rest)
    try:
        auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
        auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
        # Tính AUC weighted tự tính theo trọng số mẫu của từng lớp
        supports = np.bincount(y_test_classes, minlength=5)
        auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
    except Exception as e:
        print(f"Lỗi khi tính AUC: {e}")
        auc_macro = np.nan
        auc_per_class = [np.nan] * 5
        auc_weighted = np.nan

    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "accuracy": accuracy,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "precision_weighted": precision_weighted,
        "recall_weighted": recall_weighted,
        "f1_weighted": f1_weighted,
        "auc_macro": auc_macro,
        "auc_weighted": auc_weighted,
        "auc_per_class": auc_per_class,
        "confusion_matrix": conf_matrix
    })
    
    # In kết quả chi tiết
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test_classes),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))

    print("\n=== Macro Averages & Accuracy ===")
    print(f"Macro Precision: {precision_macro:.4f}")
    print(f"Macro Recall: {recall_macro:.4f}")
    print(f"Macro F1-Score: {f1_macro:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    
    print("\n=== Weighted Averages ===")
    print(f"Weighted Precision: {precision_weighted:.4f}")
    print(f"Weighted Recall: {recall_weighted:.4f}")
    print(f"Weighted F1-Score: {f1_weighted:.4f}")
    
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {auc_macro:.4f}")
    print(f"AUC Weighted: {auc_weighted:.4f}")
    print(f"AUC per Label: {auc_per_class}")
    
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test_classes), columns=np.unique(y_test_classes)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [13]:
process_week(1, best_params, results)


=== Processing Week 1 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.3510 - loss: 0.2237 - val_accuracy: 0.5682 - val_loss: 0.2673
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.4924 - loss: 0.1921 - val_accuracy: 0.7129 - val_loss: 0.2017
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5160 - loss: 0.1829 - val_accuracy: 0.7289 - val_loss: 0.2372
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5425 - loss: 0.1775 - val_accuracy: 0.7166 - val_loss: 0.2325
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5534 - loss: 0.1742 - val_accuracy: 0.7077 - val_loss: 0.2373
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5714 - loss: 0.1688 - val_accuracy: 0.6416 - val_loss: 0.2627
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5857 - loss: 0.1654 - val_accuracy: 0.6129 - val_loss: 0.2699
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5865 - loss: 0.1655 - val_accura

In [14]:
process_week(2, best_params, results)


=== Processing Week 2 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.4639 - loss: 0.1981 - val_accuracy: 0.6401 - val_loss: 0.2454
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5593 - loss: 0.1637 - val_accuracy: 0.7179 - val_loss: 0.2199
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5988 - loss: 0.1557 - val_accuracy: 0.7169 - val_loss: 0.1794
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6246 - loss: 0.1448 - val_accuracy: 0.5962 - val_loss: 0.2083
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6593 - loss: 0.1293 - val_accuracy: 0.6675 - val_loss: 0.1701
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6909 - loss: 0.1166 - val_accuracy: 0.6782 - val_loss: 0.2015
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.7136 - loss: 0.1042 - val_accuracy: 0.6875 - val_loss: 0.1400
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.7385 - loss: 0.0930 - val_accura

In [15]:
process_week(3, best_params, results)


=== Processing Week 3 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.4513 - loss: 0.1995 - val_accuracy: 0.6386 - val_loss: 0.2651
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.5097 - loss: 0.1807 - val_accuracy: 0.6488 - val_loss: 0.2382
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5260 - loss: 0.1754 - val_accuracy: 0.5957 - val_loss: 0.2603
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5339 - loss: 0.1728 - val_accuracy: 0.6219 - val_loss: 0.2172
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.5394 - loss: 0.1690 - val_accuracy: 0.7151 - val_loss: 0.2020
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.5683 - loss: 0.1579 - val_accuracy: 0.7189 - val_loss: 0.2058
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5725 - loss: 0.1581 - val_accuracy: 0.6692 - val_loss: 0.2189
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5958 - loss: 0.1478 - val_accura

In [16]:
process_week(4, best_params, results)


=== Processing Week 4 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.4055 - loss: 0.2127 - val_accuracy: 0.5879 - val_loss: 0.2900
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.4765 - loss: 0.1911 - val_accuracy: 0.6433 - val_loss: 0.2283
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5333 - loss: 0.1723 - val_accuracy: 0.5914 - val_loss: 0.2622
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.5552 - loss: 0.1627 - val_accuracy: 0.5922 - val_loss: 0.2478
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5709 - loss: 0.1583 - val_accuracy: 0.6054 - val_loss: 0.2610
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5881 - loss: 0.1551 - val_accuracy: 0.6767 - val_loss: 0.2146
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6177 - loss: 0.1425 - val_accuracy: 0.6667 - val_loss: 0.2064
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6250 - loss: 0.1400 - val_accur

In [17]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Accurancy: {result['accuracy']}")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Macro Precision: {result['precision_macro']}")
    print(f"  Macro Recall: {result['recall_macro']}")
    print(f"  Macro F1-Score: {result['f1_macro']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {auc_macro:.4f}")
    print(f"AUC Weighted: {auc_weighted:.4f}")
    print(f"AUC per Label: {auc_per_class}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 454.29 seconds
  Test Time: 0.69 seconds
  Accurancy: 0.7310975609756097
  Precision: [0.67010309 0.22580645 0.46153846 0.52898551 0.90465116]
  Recall: [0.69333333 0.51851852 0.58252427 0.6952381  0.77567298]
  F1-Score: [0.68152031 0.31460674 0.51502146 0.60082305 0.83521202]
  Macro Precision: 0.5582169351943889
  Macro Recall: 0.6530574399982874
  Macro F1-Score: 0.5894367168467299
  Confusion Matrix:
[[260  27  30   2  56]
 [ 17  28   1   2   6]
 [ 28   3  60   5   7]
 [  4  10   5  73  13]
 [ 79  56  34  56 778]]

=== AUC-ROC ===
AUC Macro: 0.9359
AUC Weighted: 0.9747
AUC per Label: [0.95215917 0.87125323 0.93022039 0.92781687 0.99824299]
Week 2:
  Train Time: 450.37 seconds
  Test Time: 0.69 seconds
  Accurancy: 0.8969512195121951
  Precision: [0.78807947 0.86666667 0.71428571 0.84337349 0.98257081]
  Recall: [0.952      0.96296296 0.87378641 0.66666667 0.89930209]
  F1-Score: [0.86231884 0.9122807  0.7860262  0.744680